###Create a program to build and train an Artificial Neural Network

In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


* Keras is a high-level API for building and training deep learning models. It is designed to be user-friendly, modular, and extensible, allowing for easy and rapid prototyping of deep learning models.
* TensorFlow is an end-to-end open source platform for machine learning

In [29]:
# Load the dataset
file_path = '/content/Titaanic Datasetforlogisticregression .csv'
df = pd.read_csv(file_path)

In [30]:
df

,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
882,0,2,male,27.0,0,0,13.0000
883,1,1,female,19.0,0,0,30.0000
884,0,3,female,7.0,1,2,23.4500
885,1,1,male,26.0,0,0,30.0000


**Independent Variable**

In [50]:
X = df[['Pclass', 'Sex', 'Age', 'Fare']]

In [51]:
X

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
...,...,...,...,...
882,2,male,27.0,13.0000
883,1,female,19.0,30.0000
884,3,female,7.0,23.4500
885,1,male,26.0,30.0000


**Dependent Variable**

In [52]:
y = df[['Survived']]

In [53]:
y

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
882,0
883,1
884,0
885,1


In [54]:
# Handling missing values (if any)
X['Age'].fillna(X['Age'].median(), inplace=True)
X['Fare'].fillna(X['Fare'].median(), inplace=True)
X['Sex'].fillna(X['Sex'].mode()[0], inplace=True)

<ipython-input-54-b117fff32e91>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'].fillna(X['Age'].median(), inplace=True)
<ipython-input-54-b117fff32e91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fare'].fillna(X['Fare'].median(), inplace=True)
<ipython-input-54-b117fff32e91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'].fillna(X['Sex'].mode()[0], inplace=True)


In [55]:
# Preprocessing pipeline
numeric_features = ['Age', 'Fare']
categorical_features = ['Pclass', 'Sex']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X = preprocessor.fit_transform(X)

# Adding polynomial features
poly = PolynomialFeatures(degree=2, interaction_only=True)
X = poly.fit_transform(X)

In [56]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* test_size=0.2: This specifies that 20% of the data should be used for testing, while the remaining 80% will be used for training.
* random_state=42: This ensures reproducibility by setting a seed for the random number generator.

In [59]:
 #Build the ANN model
model = Sequential()
model.add(Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

Standardize the data for Improved Model Performance

* A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
* units=5: Specifies the number of neurons in this layer. In this case, the layer has 5 neurons.
* The ReLU (Rectified Linear Unit) activation function is used here, which introduces non-linearity into the model and helps it learn complex patterns.

*  The Adam optimizer is an adaptive learning rate optimization algorithm that's widely used for training deep learning models.
* 0.001: This specifies the learning rate for the optimizer.
*




In [61]:
# Compile the model
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=400, batch_size=16, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7761 - loss: 0.4752 - val_accuracy: 0.8169 - val_loss: 0.4380
Epoch 2/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7997 - loss: 0.4732 - val_accuracy: 0.8169 - val_loss: 0.4418
Epoch 3/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8132 - loss: 0.4122 - val_accuracy: 0.8169 - val_loss: 0.4419
Epoch 4/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7766 - loss: 0.4722 - val_accuracy: 0.8169 - val_loss: 0.4415
Epoch 5/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7917 - loss: 0.4182 - val_accuracy: 0.8169 - val_loss: 0.4405
Epoch 6/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8334 - loss: 0.4000 - val_accuracy: 0.8169 - val_loss: 0.4392
Epoch 7/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8339 - loss: 0.4144 - val_accuracy: 0.8169 - val_loss: 0.4371
Epoch 8/400
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7823 - loss: 0.4155 - val_accuracy: 0.8169 - v

**INFERENCE:**
* The model achieved a test accuracy of approximately 76.40%. This means that the model correctly classified about 76.40% of the test data samples.
* The test loss is 0.5889. The loss value represents how well or poorly the model's predictions align with the actual labels.
* Lower loss indicates better performance, although the acceptable range of loss values depends on the specific problem and dataset.